<a href="https://colab.research.google.com/github/sabre-code/text-summarisation/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install transformers[torch]
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version='3.0.0')

In [3]:
from transformers import AutoTokenizer

checkpoint = "sabre-code/pegasus-large-cnn-dailymail"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
type(dataset['article'])

list

In [11]:
dta2 = dataset.select(range(10000,11000))

In [12]:
def preprocess_function(examples):

    model_inputs = tokenizer(examples["article"], max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=250, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dta2.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
dta2

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1000
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:

import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:

#torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

for param in model.model.encoder.parameters():
      param.requires_grad = False

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="pegasus-large-cnn-dailymail",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16 = True,
    logging_steps = 16,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()